<a href="https://colab.research.google.com/github/kevint2007/solarstressindez/blob/main/Kenya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee
import geemap

In [ ]:

PROJECT_ID = 'hi' # @param {type:"string"}
if not PROJECT_ID:
    raise ValueError("Please enter your own Google Cloud Project ID in the field above before running this cell.")
ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

print(f"Successfully initialized Earth Engine with project: {PROJECT_ID}")

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# --- Phase 0: Define Your 5 Clusters ---

# 1. Define the 5 Center Points (Lat/Lon estimated from your map)
# Format: [Longitude, Latitude]
eldoret_point = ee.Geometry.Point([35.2698, 0.5143])   # Cluster 1: Eldoret
embu_point    = ee.Geometry.Point([37.4500, -0.5300])  # Cluster 2: Embu
tsavo_point   = ee.Geometry.Point([38.5500, -3.4000])  # Cluster 3: The "Giant" (Voi/Taita)
kilifi_point  = ee.Geometry.Point([39.8500, -3.6300])  # Cluster 4: Mombasa North
kwale_point   = ee.Geometry.Point([39.4500, -4.1800])  # Cluster 5: Mombasa South

# 2. Create "Market Areas" (10km Radius buffers around points)
# We do this so we aren't just analyzing a single pixel, but the whole town.
cluster_1 = eldoret_point.buffer(10000)
cluster_2 = embu_point.buffer(10000)
cluster_3 = tsavo_point.buffer(10000)
cluster_4 = kilifi_point.buffer(10000)
cluster_5 = kwale_point.buffer(10000)

# 3. Combine them into a single "Feature Collection" (Like a Shapefile)
my_clusters = ee.FeatureCollection([
    ee.Feature(cluster_1, {'name': 'Eldoret_Cluster'}),
    ee.Feature(cluster_2, {'name': 'Embu_Cluster'}),
    ee.Feature(cluster_3, {'name': 'Tsavo_Cluster'}),
    ee.Feature(cluster_4, {'name': 'Mombasa_North'}),
    ee.Feature(cluster_5, {'name': 'Mombasa_South'})
])

# 4. Add them to the map to verify
# We paint them RED to match your Project 1 design
Map.centerObject(my_clusters, 7)
Map.addLayer(my_clusters, {'color': 'red'}, 'My 5 Priority Clusters')

# 5. Display the map
Map

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# Setup the Time Range (3 Years of Data)
start_date = '2021-01-01'
end_date = '2023-12-31'

# 2. Define the Sentinel-2 Collection
def get_clean_s2(roi):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterDate(start_date, end_date)
          .filterBounds(roi)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
          # Add a 'date' property to every image so we can track it later
          .map(lambda img: img.set('date', img.date().format('YYYY-MM-dd'))))
    return s2

# 3. Define the Function to Calculate NDVI & Extract Mean Value
def add_ndvi_and_reduce(img):
    # Calculate NDVI: (NIR - Red) / (NIR + Red)
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Calculate the MEAN NDVI inside the clusters
    # We use 'reduceRegions' to calculate statistics for ALL 5 clusters at once
    stats = ndvi.reduceRegions(
        collection=my_clusters,
        reducer=ee.Reducer.mean(),
        scale=100  # 100m scale is faster and sufficient for regional trends
    )

    # The result is a FeatureCollection. We attach the image date to every row.
    return stats.map(lambda f: f.set('date', img.date().format('YYYY-MM-dd')))

# 4. Run the Pipeline (This sends the instruction to Google)
print("Querying Google Earth Engine... (This might take 45-60 seconds)")

# Get the images
s2_collection = get_clean_s2(my_clusters.geometry())

# Apply the math to every image and flatten the result into one big list
all_stats = s2_collection.map(add_ndvi_and_reduce).flatten()

# 5. Convert to Pandas DataFrame (The "Download" step)
# FIXED: used 'ee_to_df' instead of the old 'ee_to_pandas'
df = geemap.ee_to_df(all_stats)

# 6. Clean and Inspect the Data
# Select only the columns we need
df = df[['date', 'mean', 'name']]

# Convert the 'date' column to datetime objects
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')
df = df.dropna() # Remove empty rows

print("Success! Data Extracted.")
print(df.head())

In [ ]:
# --- Phase 2: Visualizing the "Solar Stress Index" ---
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Setup the Plot Style
sns.set_theme(style="whitegrid")
plt.figure(figsize=(14, 7))

# 2. Draw the Lines (One color per Cluster)
# 'hue' automatically separates your 5 clusters into different colors
sns.lineplot(data=df, x='date', y='mean', hue='name', linewidth=2, alpha=0.8)

# 3. Add the "Danger Zone" Line
# NDVI < 0.3 usually indicates bare soil or extreme drought
plt.axhline(y=0.3, color='red', linestyle='--', linewidth=2, label='Critical Stress Threshold (0.3)')

# 4. Polish the Chart labels
plt.title('Agricultural Resilience (NDVI) - 2021 to 2023', fontsize=16, fontweight='bold')
plt.ylabel('Vegetation Health (Proxy for Income)', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.legend(title='Cluster Location', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# 5. Show it
plt.show()

In [ ]:
def get_rainfall(roi):
    chirps = (ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
              .filterDate(start_date, end_date)
              .filterBounds(roi))
    return chirps

def calc_rainfall(img):
    # CHIRPS has a band called 'precipitation'
    # We calculate the MEAN rainfall in the cluster (mm per 5 days)
    # scale=5000 is CRITICAL because CHIRPS pixels are 5km wide
    stats = img.reduceRegions(
        collection=my_clusters,
        reducer=ee.Reducer.mean(),
        scale=5000
    )
    return stats.map(lambda f: f.set('date', img.date().format('YYYY-MM-dd')))

rain_collection = get_rainfall(my_clusters.geometry())
rain_stats = rain_collection.map(calc_rainfall).flatten()

# 4. Convert to Pandas
# Uses the updated geemap syntax
df_rain = geemap.ee_to_df(rain_stats)

# 5. Clean Data
df_rain = df_rain[['date', 'mean', 'name']]
df_rain['date'] = pd.to_datetime(df_rain['date'])
df_rain = df_rain.sort_values('date')

# 6. Visualize Rain vs. Vegetation
# We focus on TSAVO (The "Crash" Cluster) to see the correlation
target_cluster = 'Tsavo_Cluster'

# Filter data for just Tsavo
tsavo_ndvi = df[df['name'] == target_cluster]
tsavo_rain = df_rain[df_rain['name'] == target_cluster]

# Plotting (Dual Axis)
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot Rainfall (Bars) on Left Axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Rainfall (mm)', color=color)
ax1.bar(tsavo_rain['date'], tsavo_rain['mean'], color=color, alpha=0.3, label='Rainfall', width=5)
ax1.tick_params(axis='y', labelcolor=color)

# Plot NDVI (Line) on Right Axis
ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
color = 'tab:red'
ax2.set_ylabel('Vegetation Health (NDVI)', color=color)
ax2.plot(tsavo_ndvi['date'], tsavo_ndvi['mean'], color=color, linewidth=3, label='Vegetation (NDVI)')
ax2.tick_params(axis='y', labelcolor=color)
ax2.axhline(y=0.3, color='black', linestyle='--', label='Stress Threshold')

plt.title(f'The "Lag Effect": Rain vs. Income in {target_cluster}', fontsize=16)
fig.tight_layout()
plt.show()

In [ ]:
# --- Phase 3: Merging the Data for Machine Learning ---

# 1. Prepare the Tables
# We rename columns so they don't conflict (e.g., 'mean' -> 'NDVI', 'mean' -> 'Rain')
ndvi_clean = df.rename(columns={'mean': 'NDVI'})[['date', 'name', 'NDVI']]
rain_clean = df_rain.rename(columns={'mean': 'Rain'})[['date', 'name', 'Rain']]

# 2. Merge them on 'Date' and 'Name'
# This aligns the Rain and Vegetation for the same day and location
master_df = pd.merge_asof(
    ndvi_clean.sort_values('date'),
    rain_clean.sort_values('date'),
    on='date',
    by='name',
    direction='nearest', # Match the closest rainfall reading to the satellite photo
    tolerance=pd.Timedelta('5 days') # Allow a 5-day window match
)

# 3. Handle Missing Data
master_df = master_df.dropna()

print("Success! Master Dataset Created.")
print(master_df.head())

In [ ]:
# --- Phase 4.3: Experiment B - The "Reality Check" (Temporal Split) ---
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import pandas as pd

# (Assuming Steps 0, 1, and 2 from your previous code are already run,
# meaning `cluster_data` is defined and sorted by date)

# 1. Define the Cutoff Date for the "Black Swan" Event
# We train on everything BEFORE 2022, and test on 2022 onwards
cutoff_date = pd.to_datetime('2022-01-01')

# 2. Split the Data Chronologically (NO random splitting)
train_data = cluster_data[cluster_data['date'] < cutoff_date]
test_data = cluster_data[cluster_data['date'] >= cutoff_date]

# 3. Define Features & Target for both sets
features = ['Past_30d_Rain_Avg', 'Prev_NDVI', 'Month']

X_train = train_data[features]
y_train = train_data['NDVI']

X_test = test_data[features]
y_test = test_data['NDVI']

# 4. Train Random Forest (Strictly on Past Data)
model = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42)
model.fit(X_train, y_train.values.ravel())

# 5. Predict & Score (Testing on the Unknown Future)
y_pred = model.predict(X_test)
score = r2_score(y_test, y_pred)

print(f"--- EXPERIMENT B: THE PLOT TWIST (Temporal Split) ---")
print(f"Training Period: {train_data['date'].min().date()} to {train_data['date'].max().date()}")
print(f"Testing Period:  {test_data['date'].min().date()} to {test_data['date'].max().date()}")
print(f"R² Score: {score:.3f}")
# Expected Result: A negative number (Model is worse than guessing the historical average)

# 6. Visualize the Failure for the Medium Article (Figure 3)
plt.figure(figsize=(14, 6))

# Plot Actual vs Predicted using actual Dates on the X-axis
plt.plot(test_data['date'], y_test, label='Actual NDVI (The Collapse)', color='tab:green', linewidth=2.5)
plt.plot(test_data['date'], y_pred, label='Predicted NDVI (Model Guess)', color='tab:red', linestyle='--', linewidth=2.5)

# Add the 0.3 Stress Threshold line for visual context
plt.axhline(y=0.3, color='black', linestyle=':', linewidth=2, label='Stress Threshold (0.3)')

plt.title(f'The Plot Twist: Model Failure During 2022-2023 Drought ({target_cluster})', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Vegetation Health (NDVI)', fontsize=12)
plt.legend(loc='lower left', fontsize=12)
plt.grid(True, alpha=0.3)

# Use tight_layout so it exports perfectly for Medium
plt.tight_layout()

# Save the plot locally to upload to Medium
plt.savefig('temporal_split_failure.png', dpi=300, bbox_inches='tight')
plt.show()
